# Final

In [1]:
import secrets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from cvxopt import matrix
from cvxopt import solvers

from pathlib import Path
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics.pairwise import euclidean_distances

solvers.options['show_progress'] = False
# solvers.options['glpk'] = dict(msg_lev='GLP_MSG_OFF')
sns.set_context("paper")

In [2]:
data_path = Path().resolve().parent / "data"

In [3]:
# Random seed for reproducibility
# secrets.randbits(128) # 208905213533139122735706682150229709525
rng = np.random.default_rng(208905213533139122735706682150229709525)
indices_train = rng.choice(5000, 500, replace=False)
indices_test = rng.choice(800, 500, replace=False)
flag_full_dataset = False  # If it is True it will use full train and test datasets

In [4]:
# train_list = []  # Auxiliary list of train datasets
# for f in data_path.glob("train*.txt"):
#     # Sample or full dataset
#     raw_data = np.loadtxt(f) if flag_full_dataset else np.loadtxt(f)[indices_train, :]
#     target = raw_data[:, [0]]  # Target values, i.e. digit
#     features = raw_data[:, 1:] / 255
#     train_list.append(np.hstack((target, features)))  # Add to the temp list
# train_data = np.vstack(train_list)  # Concatenate train datasets
# train_data.shape

In [5]:
# # Similar to train dataset
# test_list = []
# for f in data_path.glob("test*.txt"):
#     raw_data = np.loadtxt(f) if flag_full_dataset else np.loadtxt(f)[indices_test, :]
#     target = raw_data[:, [0]]
#     features = raw_data[:, 1:] / 255
#     test_list.append(np.hstack((target, features)))
# test_data = np.vstack(test_list)
# test_data.shape

In [6]:
# # Split datasets into features matrices and target vectors
# X_train = train_data[:, 1:]
# y_train = train_data[:, 0].astype(int)
# X_test = test_data[:, 1:]
# y_test = test_data[:, 0].astype(int)

## 1.

In [7]:
def get_data(labels, n_train_label, n_test_label, rng):
    train_list = []  # Auxiliary list of train datasets
    for f_train in data_path.glob("train*.txt"):
        if f_train.stem.removeprefix("train") not in map(str, labels):
            continue
        raw_train = np.loadtxt(f_train)
        if n_train_label is not None:
            indices_train = rng.choice(raw_train.shape[0], n_train_label, replace=False)
            raw_train = raw_train[indices_train, :]
        target_train = raw_train[:, [0]]  # Target values, i.e. digit
        features_train = raw_train[:, 1:] / 255
        train_list.append(np.hstack((target_train, features_train)))
    train_data = np.vstack(train_list)  # Concatenate train datasets

    test_list = []
    for f_test in data_path.glob("test*.txt"):
        if f_test.stem.removeprefix("test") not in map(str, labels):
            continue
        raw_test = np.loadtxt(f_test)
        if n_test_label is not None:
            indices_test = rng.choice(raw_test.shape[0], n_test_label, replace=False)
            raw_test = raw_test[indices_test, :]
        target_test = raw_test[:, [0]]
        features_test = raw_test[:, 1:] / 255
        test_list.append(np.hstack((target_test, features_test)))
    test_data = np.vstack(test_list)
    X_train = train_data[:, 1:]
    y_train = train_data[:, 0].astype(int)
    X_test = test_data[:, 1:]
    y_test = test_data[:, 0].astype(int)
    return X_train, y_train, X_test, y_test

In [8]:
def radial_basis(X1, X2, gamma):
    K = np.exp(-gamma * euclidean_distances(X1, X2,squared=True))
    return K

In [9]:
labels = np.array([3, 6])
n_train_label = 500
n_test_label = 500
X_train, y_train, X_test, y_test = get_data(labels, n_train_label, n_test_label, rng)

In [10]:
# y_i \in {-1, 1}
min_label = np.min(labels)
y_train_bin = np.where(y_train == min_label, -1, 1)
y_test_bin = np.where(y_test == min_label, -1, 1)

In [11]:
# n_train = y_train.size
# n_test = y_test.size
# K_train = radial_basis(X_train, X_train, gamma)
# M = matrix(np.outer(y_train_bin, y_train_bin) * K_train)
# e = matrix(np.ones(shape=(n_train, 1), dtype=float))
# G = matrix(np.identity(n=n_train, dtype=float))
# h = matrix(C * e)
# A = matrix(y_train_bin.reshape(1, -1).astype(float))
# b = matrix(0.0)

In [12]:
# sol = solvers.qp(M, e, G, h, A, b)
# alpha = np.array(sol["x"]).flatten()
# I = np.argwhere(alpha > 0).flatten()
# alpha_b_idx = np.argwhere((0 < alpha) & (alpha < C)).flatten()
# b = np.sum(y_train_bin[I] *  alpha[I] * K_train[I, alpha_b_idx[0]])
# K_test = radial_basis(X_train, X_test, gamma)
# class_number = np.sum(y_train_bin[I, np.newaxis] *  alpha[I, np.newaxis] * K_test[I, :], axis=0) - b * np.ones(shape=(n_test))
# y_pred_bin = np.where(class_number > 0, 1, -1)
# np.sum(y_pred_bin == y_test_bin) / n_test

In [13]:
class SVM_binary():
    def __init__(self, C, gamma):
        self.C = C
        self.gamma = gamma

    def train(self, X_train, y_train):
        n_train = X_train.shape[0]
        self.X_train = X_train
        self.y_train = y_train
        K_train = radial_basis(X_train, X_train, self.gamma)
        M = matrix(np.outer(y_train, y_train) * K_train)
        e = matrix(np.ones(shape=(n_train, 1), dtype=float))
        G = matrix(np.identity(n=n_train, dtype=float))
        h = matrix(C * e)
        A = matrix(y_train.reshape(1, -1).astype(float))
        b = matrix(0.0)
        self.sol = solvers.qp(M, e, G, h, A, b)
        self.alpha = np.array(self.sol["x"]).flatten()
        self.I = np.argwhere(self.alpha > 0).flatten()
        alpha_b_idx = np.argwhere((0 < self.alpha) & (self.alpha < C)).flatten()[0]
        self.b = np.sum(
            y_train[self.I] *  self.alpha[self.I] * K_train[self.I, alpha_b_idx]
        )

    def predict(self, X_test):
        n_test = X_test.shape[0]
        K_test = radial_basis(self.X_train, X_test, self.gamma)
        class_number = (
            np.sum(
                self.y_train[self.I, np.newaxis]
                * self.alpha[self.I, np.newaxis]
                * K_test[self.I, :],
                axis=0
            )
            - self.b * np.ones(shape=(n_test))
        )
        y_pred = np.where(class_number > 0, 1, -1)
        return y_pred

    def accuracy(self, X_test, y_test):
        y_pred = self.predict(X_test)
        return np.mean(y_pred == y_test)

In [14]:
gamma = 0.03
C = 100
svm_36 = SVM_binary(C=C, gamma=gamma)
svm_36.train(X_train, y_train_bin)
svm_36.accuracy(X_test, y_test_bin)

0.581